In [1]:
import os
import sys
import pandas as pd
from typing import List

In [2]:
!pip install beautifultable
from beautifultable import BeautifulTable

In [3]:
!pip install opencv-python
!pip install camelot-py
!pip install ghostscript
!apt-get install ghostscript

import camelot

In [18]:
#file_path="/content/drive/MyDrive/Colab Notebooks/OTL_edited.csv"
file_path="/content/drive/MyDrive/Colab Notebooks/World_Billionaires_Wikipedia.pdf"

In [29]:
# use camelot to parse tables
def get_tables(path: str, pages: List[int]):
    for page in pages:
        table_list = camelot.read_pdf(path, pages=str(page))
        if table_list.n>0:
            for tab in range(table_list.n):

                # Conversion of the the tables into the dataframes.
                table_df = table_list[tab].df
                print(table_df)

                table_df = (
                    table_df.rename(columns=table_df.iloc[0])
                    .drop(table_df.index[0])
                    .reset_index(drop=True)
                )

                table_df = table_df.apply(lambda x: x.str.replace('\n',''))

                # Change column names to be valid as XML tags
                table_df.columns = [col.replace('\n', ' ').replace(' ', '') for col in table_df.columns]
                table_df.columns = [col.replace('(', '').replace(')', '') for col in table_df.columns]

    return table_df
# extract data table from page number
df = get_tables(file_path, pages=[4])

      0                          1                 2    3  \
0   No.                       Name  Net worth\n(USD)  Age   
1     1  Bernard Arnault &\nfamily      $211 billion   74   
2     2                  Elon Musk      $180 billion   51   
3     3                 Jeff Bezos      $114 billion   59   
4     4              Larry Ellison      $107 billion   78   
5     5             Warren Buffett      $106 billion   92   
6     6                 Bill Gates      $104 billion   67   
7     7          Michael Bloomberg     $94.5 billion   81   
8     8       Carlos Slim & family       $93 billion   83   
9     9              Mukesh Ambani     $83.4 billion   65   
10   10              Steve Ballmer     $80.7 billion   67   

                                         4  \
0                              Nationality   
1                                   France   
2   South Africa\n Canada\n United\nStates   
3                           United\nStates   
4                           United\nS

In [7]:
# prepare test set
eval_df = pd.DataFrame(columns=["Data Format", "Data raw"]) # , "Question", "Answer"

# Save the data in JSON format
data_json = df.to_json(orient='records')
eval_df.loc[len(eval_df)] = ["JSON", data_json]

# Save the data as a list of dictionaries
data_list_dict = df.to_dict(orient='records')
eval_df.loc[len(eval_df)] = ["DICT", data_list_dict]

# Save the data in CSV format
csv_data = df.to_csv(index=False)
eval_df.loc[len(eval_df)] = ["CSV", csv_data]

# Save the data in tab-separated format
tsv_data = df.to_csv(index=False, sep='\t')
eval_df.loc[len(eval_df)] = ["TSV (tab-separated)", tsv_data]

# Save the data in HTML format
html_data = df.to_html(index=False)
eval_df.loc[len(eval_df)] = ["HTML", html_data]

# Save the data in LaTeX format
latex_data = df.to_latex(index=False)
eval_df.loc[len(eval_df)] = ["LaTeX", latex_data]

# Save the data in Markdown format
markdown_data = df.to_markdown(index=False)
eval_df.loc[len(eval_df)] = ["Markdown", markdown_data]

# Save the data as a string
string_data = df.to_string(index=False)
eval_df.loc[len(eval_df)] = ["STRING", string_data]

# Save the data as a NumPy array
numpy_data = df.to_numpy()
eval_df.loc[len(eval_df)] = ["NumPy", numpy_data]

# Save the data in XML format
xml_data = df.to_xml(index=False)
eval_df.loc[len(eval_df)] = ["XML", xml_data]

In [8]:
from pandas import option_context
with option_context('display.max_colwidth', 150):
    display(eval_df.head(10))

,Data Format,Data raw
0,JSON,"[{""No."":""1"",""Name"":""Bernard Arnault &family"",""NetworthUSD"":""$233\u00a0billion"",""Age"":""75"",""Nationality"":""France"",""Primarysourcesofwealth"":""LVMH""},..."
1,DICT,"[{'No.': '1', 'Name': 'Bernard Arnault &family', 'NetworthUSD': '$233 billion', 'Age': '75', 'Nationality': 'France', 'Primarysourcesofwealth': 'L..."
2,CSV,"No.,Name,NetworthUSD,Age,Nationality,Primarysourcesofwealth\n1,Bernard Arnault &family,$233 billion,75,France,LVMH\n2,Elon Musk,$195 billion,52,So..."
3,TSV (tab-separated),No.\tName\tNetworthUSD\tAge\tNationality\tPrimarysourcesofwealth\n1\tBernard Arnault &family\t$233 billion\t75\tFrance\tLVMH\n2\tElon Musk\t$195 b...
4,HTML,"<table border=""1"" class=""dataframe"">\n <thead>\n <tr style=""text-align: right;"">\n <th>No.</th>\n <th>Name</th>\n <th>NetworthU..."
5,LaTeX,\begin{tabular}{llllll}\n\toprule\nNo. & Name & NetworthUSD & Age & Nationality & Primarysourcesofwealth \\\n\midrule\n1 & Bernard Arnault &family...
6,Markdown,| No. | Name | NetworthUSD | Age | Nationality | Primarysourcesofwealth |\n|------:|:-------------...
7,STRING,No. Name NetworthUSD Age Nationality Primarysourcesofwealth\n 1 Bernard Arnault &family $233 billion 75...
8,NumPy,"[[1, Bernard Arnault &family, $233 billion, 75, France, LVMH], [2, Elon Musk, $195 billion, 52, South Africa Canada UnitedStates, Tesla, SpaceX], ..."
9,XML,<?xml version='1.0' encoding='utf-8'?>\n<data>\n <row>\n <No.>1</No.>\n <Name>Bernard Arnault &amp;family</Name>\n <NetworthUSD>$233 bil...


In [9]:
MESSAGE_SYSTEM_CONTENT = """You are a customer service agent that helps a customer with answering questions.
Please answer the question based on the provided context below.
Make sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses.
If the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer."""

In [ ]:
!pip install openai==0.28

In [11]:
# from openai import AzureOpenAI

# client = AzureOpenAI(
#     api_key=OAI_API_Key,
#     api_version=OAI_API_Version,
#     azure_endpoint=OAI_API_Base)

# def response_test(question:str, context:str, model:str = "gpt-4"):
#     response = client.chat.completions.create(
#         model=model,
#         messages=[
#             {
#                 "role": "system",
#                 "content": MESSAGE_SYSTEM_CONTENT,
#             },
#             {"role": "user", "content": question},
#             {"role": "assistant", "content": context},
#         ],
#     )

#     return response.choices[0].message.content

import openai

# Set your OpenAI API key
openai.api_key = ""

def response_test(question: str, context: str, model: str = "gpt-4"):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": MESSAGE_SYSTEM_CONTENT,
            },
            {"role": "user", "content": question},
            {"role": "assistant", "content": context},
        ],
    )

    return response.choices[0].message["content"]


In [12]:
def run_question_test(query: str, eval_df:str):

    questions = []
    answers = []

    for index, row in eval_df.iterrows():
        questions.append(query)
        response = response_test(query, str(row['Data raw']))
        answers.append(response)

    eval_df['Question'] = questions
    eval_df['Answer'] = answers

    return eval_df

def BeautifulTableformat(query:str, results:pd.DataFrame, MaxWidth:int = 250):
    table = BeautifulTable(maxwidth=MaxWidth, default_alignment=BeautifulTable.ALIGN_LEFT)
    table.columns.header = ["Data Format", "Query", "Answer"]
    for index, row in results.iterrows():
        table.rows.append([row['Data Format'], query, row['Answer']])

    return table

In [13]:
query = "What's the Elon Musk's net worth?"
result_df1 = run_question_test(query, eval_df.copy())
table = BeautifulTableformat(query, result_df1, 150)
print(table)

+---------------------+-----------------------------------+----------------------------------------+
| Data Format         | Query                             | Answer                                 |
+---------------------+-----------------------------------+----------------------------------------+
| JSON                | What's the Elon Musk's net worth? | Elon Musk's net worth is $195 billion. |
+---------------------+-----------------------------------+----------------------------------------+
| DICT                | What's the Elon Musk's net worth? | Elon Musk's net worth is $195 billion. |
+---------------------+-----------------------------------+----------------------------------------+
| CSV                 | What's the Elon Musk's net worth? | Elon Musk's net worth is $195 billion. |
+---------------------+-----------------------------------+----------------------------------------+
| TSV (tab-separated) | What's the Elon Musk's net worth? | Elon Musk's net worth is $195 b

In [14]:
query = "What are the sixth richest billionaire in 2023 and 2024 net worth?"
result_df2 = run_question_test(query, eval_df.copy())
table = BeautifulTableformat(query, result_df2, 150)
print(table)

+---------------------+----------------------------------+-------------------------------------------------------------------------------------------+
| Data Format         | Query                            | Answer                                                                                    |
+---------------------+----------------------------------+-------------------------------------------------------------------------------------------+
| JSON                | What are the sixth richest billi | The sixth richest billionaire in 2023 is Warren Buffett with a net worth of $133 billion. |
|                     | onaire in 2023 and 2024 net wort |  The information for 2024 is not available in the context provided.                       |
|                     | h?                               |                                                                                           |
+---------------------+----------------------------------+------------------------------------

In [16]:
query = "What are the sixth richest billionaire in 2021 net worth?"
result_df3 = run_question_test(query, eval_df.copy())
table = BeautifulTableformat(query, result_df3, 150)
print(table)

+---------------------+---------------------------------------+--------------------------------------------------------------------------------------+
| Data Format         | Query                                 | Answer                                                                               |
+---------------------+---------------------------------------+--------------------------------------------------------------------------------------+
| JSON                | What are the sixth richest billionair | The sixth richest billionaire in 2021 according to the information provided is Warre |
|                     | e in 2021 net worth?                  | n Buffett with a net worth of $133 billion.                                          |
+---------------------+---------------------------------------+--------------------------------------------------------------------------------------+
| DICT                | What are the sixth richest billionair | The sixth richest billionaire 